# 1. 获取文章链接
AI Base 的文章较为特殊，文章的编号是连续的，获取第一条（最新的）文章的编号即可递推获得其它文章的编号

In [1]:
import requests
from bs4 import BeautifulSoup
import re

def extract_snumber_from_url(base_url):
    try:
        response = requests.get(base_url)
        response.encoding = 'utf-8'

        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a')
        # print(links)

        # snumbers = []

        for link in links:
            href = link.get('href')
            if href:
                pattern = r'/news/(\d+)' 
                match = re.search(pattern, href)
                if match:
                    snumber = int(match.group(1))
                    return snumber
                    # snumbers.append(snumber)
        # return snumbers
    except Exception as e:
        print(f"error: {e}")
    return None

In [2]:
number = extract_snumber_from_url("https://www.aibase.com/zh/news")
print(number)

20555


# 2. 获取文章内容

根据 crawl4ai 项目的案例进行适当的改造即可读取，主要爬取文章标题、发布时间和正文三个内容。

## 测试一下 playwright的功能

浏览器文件

记得先安装 `playwright install`

In [4]:
import asyncio
from playwright.async_api import async_playwright

async def test_playwright():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        await page.goto('https://www.baidu.com')
        title = await page.title()
        print(f"页面标题: {title}")
        await browser.close()

# 在 Jupyter 中运行
await test_playwright()


页面标题: 百度一下，你就知道


### 上面的提取网页方法都失败了，对网页重新分析，通过分析加载 javascript 后的网页内容进行提取

以下方法是成功的

In [5]:
import json
import asyncio
import re
from crawl4ai import AsyncWebCrawler

async def extract_ai_news_article(article_id):
    """使用正则表达式提取AIbase新闻文章数据"""
    
    async with AsyncWebCrawler(verbose=False) as crawler:
        result = await crawler.arun(
            url=f"https://www.aibase.com/zh/news/{article_id}",
            bypass_cache=True,
            js_code=["await new Promise(resolve => setTimeout(resolve, 5000));"],
            timeout=30000,
        )

        if not result.success:
            print(f"页面爬取失败: {result.error}")
            return None

        html = result.html
        
        # 提取标题
        title_match = re.search(r'<h1[^>]*>(.*?)</h1>', html, re.DOTALL | re.IGNORECASE)
        title = title_match.group(1).strip() if title_match else ""
        
        # 提取内容 - 查找post-content类
        content_pattern = r'<div[^>]*class="[^"]*post-content[^"]*"[^>]*>(.*?)</div>'
        content_match = re.search(content_pattern, html, re.DOTALL | re.IGNORECASE)
        
        if content_match:
            content_html = content_match.group(1)
            # 移除HTML标签
            content = re.sub(r'<[^>]+>', '', content_html)
            content = re.sub(r'\s+', ' ', content).strip()
        else:
            content = ""
        
        # 查找日期信息
        date_patterns = [
            r'(\d+)\s*分钟阅读',
            r'(Oct\s+\d+,\s+\d+)',
            r'(\d{4}-\d{2}-\d{2})',
        ]
        
        publish_date = ""
        for pattern in date_patterns:
            match = re.search(pattern, html)
            if match:
                publish_date = match.group(1)
                break
        
        # 提取分类信息
        category = ""
        if "AI新闻资讯" in html:
            category = "AI新闻资讯"
        
        data = {
            "title": title,
            "content": content,
            "publish_date": publish_date,
            "category": category,
            "content_length": len(content)
        }
        
        return data

# 使用示例
async def main():
    print("开始提取文章数据...")
    
    result = await extract_ai_news_article(number)  #提取当前最新的一条咨询
    
    if result:
        print("提取成功!")
        print(json.dumps(result, indent=2, ensure_ascii=False))
    else:
        print("提取失败")

# 运行
await main()


开始提取文章数据...


[INIT].... → Crawl4AI 0.7.3 

[FETCH]... ↓ https://www.aibase.com/zh/news/20555                                                                 |
✓ | ⏱: 5.03s 

[SCRAPE].. ◆ https://www.aibase.com/zh/news/20555                                                                 |
✓ | ⏱: 0.02s 

[COMPLETE] ● https://www.aibase.com/zh/news/20555                                                                 |
✓ | ⏱: 5.06s 

提取成功!
{
  "title": "​Nvidia 投资 5 亿美元，Cohere 公司估值达 68 亿美元",
  "content": "根据最近的报道，Cohere 公司今天宣布成功完成一轮5亿美元的融资。这轮融资由 Radical Ventures 和 Inovia Capital 共同牵头，参与者还包括 Nvidia、AMD Ventures、Salesforce Ventures 等多个机构投资者。此次融资使得 Cohere 的估值从此前的50亿美元增至68亿美元。图源备注:图片由AI生成，图片授权服务商MidjourneyCohere 总部位于多伦多，是一家专注于开发企业级大型语言模型（LLM）的公司。该公司推出了一系列名为 “Command” 的模型，其中最强大的 “Command A” 于今年三月首次亮相。Cohere 声称，其模型在多项任务上能够匹配 GPT-4的性能，并且响应速度提高了75%。除了语言模型，Cohere 还提供嵌入模型，这是一种将文件转化为神经网络可以理解的紧凑数值形式的技术。Cohere 表示，其最新的嵌入模型能够处理长度达到200页的多模态记录和文档。企业用户可以通过云端 API 访问 Cohere 的模型，或选择在本地部署，后者还可选择使用 “隔离配置”。这种配置的 LLM 只能在内部基础设施上运行，无法通过互联网访问。Cohere 近期也显示出强劲的需求增长。据《信息周刊》报道，该公司预计到年底年化收入将超过2亿美元，这比2月份的数字几乎翻了一番。在宣布融资的同时，Cohere 还任命了两位高管。知名机器学习研究者 Joelle Pineau 成为新的首席人工智能官，而前 Uber 高管 Francois Chadwick 则担任首席技术官。Pineau 曾领导 Meta 的 FAIR 机器学习实验室，她表示将把更多研究力量集中在上周推出的 AI 生产力平台 North 上。该平台利用 AI 代理自动化数据同步等任务。Pineau 还透露，Cohere 计划扩大其 AI 开发团队，或将从 Meta 招募机器学习研究员。Francois Chadwick 的任命暗示 Cohere 可能为未来的上市做准备。在短期内，Cohere 可能会利用新募集的资金进行收购。此次融资发生在 Cohere 收购位于温哥华的 AI 公司 

# 3. 开始提取ai咨询内容

需要注意的是：

因为第一部分中获取的是当天最新咨询的编号，我们第一次只要手动获取24小时前最后的那个编号即可


In [6]:
# 8.14  24小时前最后一条咨询编号为 20448，当天下午再次更新是20482, 当前最新的编号是 `number`
# 8.15  24小时前最后一条咨询编号为 20507
# 8.16  24小时前最后一条咨询编号为 20516

import json
from datetime import datetime
import os

# 获取当前时间并格式化为文件名
current_time = datetime.now().strftime("%Y_%m_%d")
file_name = f"{current_time}.json"

# （`yesterday_last_number` 也是需要每天更改的 ）
yesterday_last_number = 20517
all_news_data = []  # 添加这行来存储所有数据

for i in range(yesterday_last_number, number + 1):
    print(f"正在提取咨询编号: {i}")
    article_data = await extract_ai_news_article(i)
    
    if article_data:  # 添加这个检查
        article_data['news_id'] = i  # 添加新闻ID
        all_news_data.append(article_data)  # 添加到列表中

# 在循环结束后添加保存代码（需要更改为每天的日期作为文件名）
with open(file_name, 'w', encoding='utf-8') as f:
    json.dump(all_news_data, f, ensure_ascii=False, indent=2)

print(f"已保存 {len(all_news_data)} 条新闻到 news_data.json")


正在提取咨询编号: 20517


[INIT].... → Crawl4AI 0.7.3 

[FETCH]... ↓ https://www.aibase.com/zh/news/20517                                                                 |
✓ | ⏱: 0.82s 

[SCRAPE].. ◆ https://www.aibase.com/zh/news/20517                                                                 |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://www.aibase.com/zh/news/20517                                                                 |
✓ | ⏱: 0.83s 

...

[INIT].... → Crawl4AI 0.7.3 

[FETCH]... ↓ https://www.aibase.com/zh/news/20555                                                                 |
✓ | ⏱: 0.78s 

[SCRAPE].. ◆ https://www.aibase.com/zh/news/20555                                                                 |
✓ | ⏱: 0.01s 

[COMPLETE] ● https://www.aibase.com/zh/news/20555                                                                 |
✓ | ⏱: 0.80s 

已保存 39 条新闻到 news_data.json


# 4. 将提取好的json数据做摘要

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()
api_key=os.getenv("ZHIPU_API_KEY")

def get_news_summary(data):
    
    client = OpenAI(
        api_key=api_key,
        base_url="https://open.bigmodel.cn/api/paas/v4/"
    )

    system_prompt = """
    ## Goals
    读取并解析JSON格式的文章，提炼出文章的主旨，形成一句简洁的概述。

    ## Constrains:
    概述长度150字左右，保持文章的原意和重点。

    ## Skills
    JSON解析能力，文章内容理解和总结能力。

    ## Output Format
    一句话概述，简洁明了，不超过150字。

    ## Workflow:
    1. 读取并解析JSON格式的文章
    2. 理解文章内容，提取关键信息
    3. 生成一句简洁的概述，不超过80字
    """

    try:
        response = client.chat.completions.create(
            model="glm-4.5-flash",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"文章内容：{data}"}
            ],
            top_p=0.7,
            temperature=0.1,
            stream=False
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"error: {e}")
        return None

In [ ]:
import json

with open(file_name, 'r', encoding='utf-8') as f:
    new_datas = json.load(f)
    # for i in range(len(new_datas)):

    summary = get_news_summary(new_datas)

print(summary)
        


1. Midjourney为标准订阅用户添加高清视频生成功能，提升创作灵活性和效率。

2. 字节跳动开源VeOmni框架，通过统一API显著提升多模态模型训练效率。

3. 特斯拉FSD完成580公里无人驾驶跨城旅程，计划推出自动充电功能。

4. 谷歌为Gemini添加记忆功能和隐私聊天模式，提升个性化服务同时保护隐私。

5. Midjourney升级标准用户功能，使其能够生成高清视频并支持批量制作。

6. Meta内部文件显示其AI聊天机器人被允许与儿童进行浪漫对话，引发争议。

7. Mistral AI发布Medium 3.1模型，在性能、语调和网络搜索智能性方面全面升级。

8. 谷歌推出"How Was This Made"功能，帮助用户识别照片和视频的真实性。

9. Multiverse Computing发布超微型AI模型，可在智能手机等设备上离线运行。

10. 香港大学与多家机构开源OpenCUA框架，帮助开发者构建个性化电脑智能助手。

11. 谷歌在Google Flights中上线Flight Deals工具，帮助用户以自然语言搜索优惠航班。

12. 昆仑万维发布Mureka V7.5模型和MoE-TTS语音合成框架，提升中文音乐创作能力。

13. OpenAI考虑在ChatGPT中引入广告，同时强调订阅模式仍有巨大增长潜力。

14. 谷歌发布Gemma3270M开源AI模型，体积小巧且能效高，支持在智能手机上离线运行。

15. Anthropic为Claude推出学习模式，采用苏格拉底式教学法引导用户思考而非直接给出答案。

16. Anthropic为Claude AI推出学习模式，强调引导学习为学生和开发者提供支持。

17. 上海交通大学推出Gen3DHF数据集和评估指标，用于评估AI生成的3D人脸质量。

18. 苹果研究团队成功训练开源模型自主学习SwiftUI用户界面开发，解决UI代码生成难题。

19. Arm推出基于AI的神经超级采样技术，提升移动设备图形质量，降低GPU工作负载。

20. HTC推出超轻量级AI眼镜VIVE Eagle，以本地化隐私保护策略挑战Meta市场主导地位。

21. AI编程独角兽Lovable计划12个月内实现10亿美元年度经常性收入，月增长达800万美元。

22. 腾讯

### 将摘要好的数据写入到一个 .txt文件中

In [ ]:
txt_file_name = f"{current_time}_news_data.txt"
with open(txt_file_name, 'w', encoding='utf-8') as f:
    f.write(summary)